In [1]:
# Document Store
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

/home/hattori-k/.conda/envs/qa/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# サンプルデータでドキュメントを作成
doc_dir = "data/tutorial3"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
document_store.write_documents(docs)

In [6]:
# TF-IDFによるRetrieverの初期化
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

# FARMReader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


Downloading: 100%|██████████| 571/571 [00:00<00:00, 262kB/s]
Downloading: 100%|██████████| 473M/473M [00:06<00:00, 75.9MB/s] 
Downloading: 100%|██████████| 79.0/79.0 [00:00<00:00, 35.3kB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 913kB/s]  
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 724kB/s] 
Downloading: 100%|██████████| 772/772 [00:00<00:00, 382kB/s]


In [7]:
# パイプラインの作成
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [10]:
# パイプラインによる質問のサンプル実行
prediction = pipe.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 99.78 Batches/s]


In [11]:
print_answers(prediction, details="minimum")


Query: Who is the father of Arya Stark?
Answers:
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's "
                   'half-brother Jon Snow gifts A'},
    {   'answer': 'Robert Baratheon',
        'context': 'hen Gendry gives it to Arya, he tells her he is the '
                   'bastard son of Robert Baratheon. Aware of their chances of '
                   'dying in the upcoming battle and Arya w'},
    {   'answer': 'Eddard and Catelyn Stark',
        'context': 'tark ===\n'
                   'Arya Stark is the third child and younger daughter of '
     